In [1]:
# Imports 
import numpy as np
import random
from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [15, 72],
 'London': [1, 72],
 'Moscow': [22, 45],
 'Barcelona': [37, 75],
 'Rome': [72, 11],
 'Paris': [42, 17],
 'Vienna': [67, 43],
 'Munich': [77, 1],
 'Istanbul': [87, 47],
 'Kyiv': [73, 60],
 'Bucharest': [32, 33],
 'Minsk': [6, 72],
 'Warsaw': [26, 83],
 'Budapest': [80, 39],
 'Milan': [1, 84],
 'Prague': [95, 68],
 'Sofia': [58, 81],
 'Birmingham': [36, 65],
 'Brussels': [76, 75],
 'Amsterdam': [72, 27]}

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Birmingham', 'Warsaw', 'Amsterdam', ..., 'Barcelona', 'London',
        'Moscow'],
       ['Amsterdam', 'Sofia', 'Vienna', ..., 'Munich', 'Bucharest',
        'Prague'],
       ['Rome', 'Prague', 'Berlin', ..., 'Milan', 'Barcelona', 'London'],
       ...,
       ['Brussels', 'Minsk', 'Birmingham', ..., 'Paris', 'Bucharest',
        'Amsterdam'],
       ['Kyiv', 'Barcelona', 'Milan', ..., 'Rome', 'Bucharest', 'Warsaw'],
       ['Sofia', 'Bucharest', 'Barcelona', ..., 'Warsaw', 'Prague',
        'Paris']], dtype='<U10')

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 962.9656404 ,  903.34883024,  862.40214294,  914.15870436,
        996.98492535,  927.3494189 ,  984.76775291, 1004.9143699 ,
        967.77804821,  899.43955608,  968.98553301,  695.61010551,
       1048.41786566,  886.92661398, 1130.47302346, 1063.90868689,
        787.69089688, 1042.75410732,  909.71981335,  950.67636957,
        848.6976465 ,  772.18643752,  905.74709534,  903.3247066 ,
        881.14311572,  886.65973758, 1036.87288509,  800.61987596,
        995.04316118,  984.90600155,  923.37176718, 1034.74770749,
        940.70514825,  795.08110711,  914.74786135,  967.55786359,
        890.44720406,  986.36370689, 1115.1594952 ,  967.61801019,
       1179.08328913,  923.24733459, 1137.39084115,  884.17002941,
       1246.4474379 ,  884.87028167, 1002.6525649 , 1088.44099748,
        905.88459129,  975.35412439,  940.96586305,  991.10998349,
        937.78165909,  977.65962312,  971.09183274,  988.68563626,
        965.74670493, 1045.71559649, 1054.73692224,  971.14517

In [7]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Kyiv', 'Vienna', 'Birmingham', 'Minsk', 'Barcelona', 'Prague',
       'Paris', 'Budapest', 'Warsaw', 'Rome', 'London', 'Moscow', 'Milan',
       'Amsterdam', 'Munich', 'Istanbul', 'Brussels', 'Berlin', 'Sofia',
       'Bucharest'], dtype='<U10')

In [8]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Moscow', 'Milan', 'Warsaw', 'Munich', 'Kyiv', 'Vienna',
       'Birmingham', 'Budapest', 'Barcelona', 'Brussels', 'London',
       'Paris', 'Rome', 'Prague', 'Berlin', 'Sofia', 'Istanbul',
       'Amsterdam', 'Minsk', 'Bucharest'], dtype='<U10')

In [9]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Moscow', 'Sofia', 'Warsaw', 'Munich', 'Brussels', 'Vienna',
       'Paris', 'Budapest', 'Rome', 'Kyiv', 'Barcelona', 'Birmingham',
       'London', 'Prague', 'Berlin', 'Milan', 'Istanbul', 'Amsterdam',
       'Minsk', 'Bucharest'], dtype='<U10')

In [10]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%50==0: print(i, best_solution[1], fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 inf 960.7479489748721 25/05/22 10:55
50 598.8877269304701 959.1479901032847 25/05/22 10:55
100 532.192692198039 959.1562570920673 25/05/22 10:55
150 532.192692198039 968.5703380011862 25/05/22 10:55
200 532.192692198039 969.8616692729005 25/05/22 10:55
250 532.192692198039 961.2650179483641 25/05/22 10:55
300 532.192692198039 977.4155994602722 25/05/22 10:55
350 532.192692198039 958.0591526944776 25/05/22 10:55
400 532.192692198039 965.3645844320209 25/05/22 10:55
450 532.192692198039 963.6161347056378 25/05/22 10:55
500 532.192692198039 972.2092964592971 25/05/22 10:55
550 532.192692198039 965.5250600258111 25/05/22 10:55
600 532.192692198039 968.3362818441161 25/05/22 10:55
650 532.192692198039 962.6928690969451 25/05/22 10:55
700 532.192692198039 960.8496283046179 25/05/22 10:55
750 532.192692198039 975.6636395022506 25/05/22 10:55
800 532.192692198039 961.0172026792596 25/05/22 10:55
850 532.192692198039 954.8220111479557 25/05/22 10:55
900 532.192692198039 973.7068774410055 25/0

7500 532.192692198039 970.4694131831665 25/05/22 10:57
7550 532.192692198039 951.0564742041253 25/05/22 10:57
7600 532.192692198039 953.4393736063455 25/05/22 10:57


KeyboardInterrupt: 

In [31]:
best_solution

[8454,
 595.921855435766,
 array([['Budapest', 'Prague', 'Rome', 'Warsaw', 'Sofia', 'Milan',
         'Vienna', 'Barcelona', 'Paris', 'Kyiv', 'Amsterdam', 'London',
         'Brussels', 'Istanbul', 'Berlin', 'Bucharest', 'Minsk',
         'Birmingham', 'Moscow', 'Munich']], dtype='<U10')]